In [ ]:
import os
import torch
import torch.nn as nn
import numpy as np
from matplotlib import pyplot as plt
from tqdm import tqdm
from torch import optim
from utils import *
from modules import UNet
import logging
from torch.utils.tensorboard import SummaryWriter
logging.basicConfig(format="%(asctime)s - %(levelname)s: %(message)s", level=logging.INFO, datefmt="%I:%M:%S")

In [ ]:
"zy" "test" "again" "test" 

In [4]:
def generate_linear_schedule(T, low, high):
    return torch.linspace(low, high, T)

def generate_cosine_schedule(T, s=0.008):
    def f(t, T):
        return (np.cos((t / T + s) / (1 + s) * np.pi / 2)) ** 2
    
    alphas = []
    f0 = f(0, T)

    for t in range(T + 1):
        alphas.append(f(t, T) / f0)
    
    betas = []

    for t in range(1, T + 1):
        betas.append(min(1 - alphas[t] / alphas[t - 1], 0.999))
    
    return torch.tensor(betas)
#betas = generate_linear_schedule(20)
betas = generate_linear_schedule(20, 1e-4, 0.002)
alpha = 1 -  betas
alpha_hat = torch.cumprod(alpha,dim=0)

In [5]:
alpha_hat

tensor([0.9999, 0.9997, 0.9994, 0.9990, 0.9985, 0.9979, 0.9972, 0.9964, 0.9955,
        0.9945, 0.9934, 0.9922, 0.9909, 0.9895, 0.9881, 0.9865, 0.9848, 0.9830,
        0.9812, 0.9792])

In [6]:
t=torch.tensor([3, 3, 3, 3, 3])
t=torch.tensor([2, 2, 2, 2, 2])

In [7]:
alpha_hat[t]
sqrt_alpha_hat = torch.sqrt(alpha_hat[t])
sqrt_one_minus_alpha_hat = torch.sqrt(1 - alpha_hat[t])
Ɛ = torch.randn_like(x)

In [8]:
sqrt_alpha_hat

tensor([0.9997, 0.9997, 0.9997, 0.9997, 0.9997])

In [3]:
x = torch.randn((5, 3, 4,4))
x

tensor([[[[-0.4683,  1.1691, -1.7642,  0.5617],
          [ 0.3483, -0.2771, -1.0007, -2.1744],
          [ 0.5816, -0.7072, -1.7514, -0.4293],
          [ 2.0705,  0.6799, -0.9155,  0.0820]],

         [[-0.9093, -0.5691,  1.6219, -1.8850],
          [ 0.3668, -0.9200,  1.4821, -0.6295],
          [-1.5128,  0.2808, -0.6758, -0.0405],
          [-0.2306, -0.7444,  0.1601, -2.8858]],

         [[ 0.9065, -0.7184, -0.1060, -0.8009],
          [ 1.1315,  0.4494, -0.9772,  1.2910],
          [-0.4895, -0.7633,  0.2498,  0.4242],
          [ 0.2720,  1.2698, -0.2295,  0.2178]]],


        [[[ 0.5923,  0.5806,  1.0161,  0.5673],
          [-2.3601, -0.4190,  0.4690, -0.1333],
          [-0.0908,  0.2838, -0.5827, -0.9176],
          [-0.3681, -0.9478,  1.6393,  1.0664]],

         [[ 0.4388, -1.1634, -0.1718,  0.5890],
          [ 0.6816, -0.2970, -1.1076,  0.4461],
          [ 0.6595,  0.3420,  1.4287, -1.7068],
          [ 1.1961, -0.8003, -1.2595, -1.5792]],

         [[ 1.1090, -0.6468,

In [11]:
sqrt_alpha_hat = torch.sqrt(alpha_hat[t])[:, None, None, None]
#sqrt_alpha_hat = torch.sqrt(alpha_hat[t]).reshape(5,1,1,1)
sqrt_one_minus_alpha_hat = torch.sqrt(1 - alpha_hat[t])[:, None, None, None]
Ɛ = torch.randn_like(x)

In [13]:
sqrt_alpha_hat.shape

torch.Size([5, 1, 1, 1])

In [110]:
sqrt_alpha_hat * x + sqrt_one_minus_alpha_hat * Ɛ

tensor([[[[ 1.3286, -0.2353,  0.0991, -1.3773],
          [-0.7290, -0.1598, -0.4245, -0.4987],
          [-0.2052, -0.4233, -0.7123, -0.0087],
          [-0.3423, -0.3240,  0.9511,  0.7970]],

         [[-0.4586,  0.3034, -0.2492,  0.0521],
          [-0.2235, -0.5247,  0.3555, -0.9259],
          [ 0.1602,  0.1333, -0.0447, -0.4147],
          [ 0.2762, -0.0651, -0.0933,  0.3236]],

         [[ 0.6375,  0.6065, -0.0302, -0.1874],
          [-0.3175,  0.3098,  0.7919,  0.8397],
          [-0.7901,  0.1536, -0.1100,  0.2060],
          [-0.9524, -0.2805,  0.7237, -0.1172]]],


        [[[-0.0116,  0.5159, -0.4975,  0.8877],
          [ 0.0821, -0.0336,  0.4018,  0.0297],
          [-0.9150,  0.3500, -0.5068, -0.4641],
          [-0.3773,  0.4975,  0.2511,  0.0233]],

         [[ 0.6040,  0.2400, -0.2427, -0.0272],
          [ 0.2798, -0.3201,  0.4901,  0.3335],
          [-0.0325, -0.1514, -0.2679,  0.4987],
          [ 0.5815,  0.2390,  0.0127, -0.4465]],

         [[-0.4399, -0.4309,

In [123]:
def sample_timesteps(n):
    return torch.randint(low=1, high=20, size=(n,))
sample_timesteps(5)

tensor([ 9,  1, 17,  7, 12])

In [ ]:
class Diffusion:
    def __init__(self, noise_steps=1000, beta_start=1e-4, beta_end=0.02, img_size=256, device="cuda"):
        self.noise_steps = noise_steps
        self.beta_start = beta_start
        self.beta_end = beta_end
        self.img_size = img_size
        self.device = device
        self.beta = self.prepare_noise_schedule().to(device)
        self.alpha = 1. - self.beta
        self.alpha_hat = torch.cumprod(self.alpha, dim=0)

    def prepare_noise_schedule(self, use_cosine=False, s=0.008):  
        if use_cosine == True:
            def f(t, self.noise_steps):
                return (np.cos((t / self.noise_steps + s) / (1 + s) * np.pi / 2)) ** 2
            alphas = []
            f0 = f(0, self.noise_steps)
            for t in range(self.noise_steps + 1):
                alphas.append(f(t, self.noise_steps) / f0)
            betas = []
            for t in range(1, self.noise_steps + 1):
                betas.append(min(1 - alphas[t] / alphas[t - 1], 0.999))
            return torch.tensor(betas)
        else:
            return torch.linspace(self.beta_start, self.beta_end, self.noise_steps)

    def noise_images(self, x, t):
        sqrt_alpha_hat = torch.sqrt(self.alpha_hat[t])[:, None, None, None]
        sqrt_one_minus_alpha_hat = torch.sqrt(1 - self.alpha_hat[t])[:, None, None, None]
        Ɛ = torch.randn_like(x)
        return sqrt_alpha_hat * x + sqrt_one_minus_alpha_hat * Ɛ, Ɛ

    def sample_timesteps(self, n):
        return torch.randint(low=1, high=self.noise_steps, size=(n,))

    def sample(self, model, n):
        logging.info(f"Sampling {n} new images....")
        model.eval()
        with torch.no_grad():
            x = torch.randn((n, 3, self.img_size, self.img_size)).to(self.device)
            for i in tqdm(reversed(range(1, self.noise_steps)), position=0):
                t = (torch.ones(n) * i).long().to(self.device)
                predicted_noise = model(x, t)
                alpha = self.alpha[t][:, None, None, None]
                alpha_hat = self.alpha_hat[t][:, None, None, None]
                beta = self.beta[t][:, None, None, None]
                if i > 1:
                    noise = torch.randn_like(x)
                else:
                    noise = torch.zeros_like(x)
                x = 1 / torch.sqrt(alpha) * (x - (beta / (torch.sqrt(1 - alpha_hat))) * predicted_noise) + torch.sqrt(beta) * noise
        model.train()
        x = (x.clamp(-1, 1) + 1) / 2
        x = (x * 255).type(torch.uint8)
        return x

In [ ]:
def train(args):
    setup_logging(args.run_name)
    device = args.device
    dataloader = get_data(args)
    model = UNet().to(device)
    optimizer = optim.AdamW(model.parameters(), lr=args.lr)
    mse = nn.MSELoss()
    diffusion = Diffusion(img_size=args.image_size, device=device)
    logger = SummaryWriter(os.path.join("runs", args.run_name))
    l = len(dataloader)

    for epoch in range(args.epochs):
        logging.info(f"Starting epoch {epoch}:")
        pbar = tqdm(dataloader)
        for i, (images, _) in enumerate(pbar):
            images = images.to(device)
            t = diffusion.sample_timesteps(images.shape[0]).to(device)
            x_t, noise = diffusion.noise_images(images, t)
            predicted_noise = model(x_t, t)
            loss = mse(noise, predicted_noise)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            pbar.set_postfix(MSE=loss.item())
            logger.add_scalar("MSE", loss.item(), global_step=epoch * l + i)

        sampled_images = diffusion.sample(model, n=images.shape[0])
        save_images(sampled_images, os.path.join("results", args.run_name, f"{epoch}.jpg"))
        torch.save(model.state_dict(), os.path.join("models", args.run_name, f"ckpt.pt"))


In [ ]:


def launch():
    import argparse
    parser = argparse.ArgumentParser()
    args = parser.parse_args()
    args.run_name = "DDPM_Uncondtional"
    args.epochs = 500
    args.batch_size = 12
    args.image_size = 64
    args.dataset_path = r"C:\Users\----------------------------"
    args.device = "cuda"
    args.lr = 3e-4
    train(args)


In [ ]:
if __name__ == '__main__':
    launch()

In [ ]:
    # device = "cuda"
    # model = UNet().to(device)
    # ckpt = torch.load("./working/orig/ckpt.pt")
    # model.load_state_dict(ckpt)
    # diffusion = Diffusion(img_size=64, device=device)
    # x = diffusion.sample(model, 8)
    # print(x.shape)
    # plt.figure(figsize=(32, 32))
    # plt.imshow(torch.cat([
    #     torch.cat([i for i in x.cpu()], dim=-1),
    # ], dim=-2).permute(1, 2, 0).cpu())
    # plt.show()